In the previous router, we invoked the model and, if it chose to call a tool, we returned a ToolMessage to the user.

But, what if we simply pass that ToolMessage back to the model?

We can let it either (1) call another tool or (2) respond directly.

This is the intuition behind ReAct, a general agent architecture.

act - let the model call specific tools
observe - pass the tool output back to the model
reason - let the model reason about the tool output to decide what to do next (e.g., call another tool or just respond directly)

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

: 

In [ ]:
from langchain_groq import ChatGroq
# llm=ChatGroq(model="qwen-2.5-32b")
llm=ChatGroq(model="deepseek-r1-distill-llama-70b")
result=llm.invoke("What's your name")
result

In [ ]:
## This will be the tools 
def add(a: int, b: int) -> int:
    """Add a and b.
    
    Args:
    a : First number
    b : Second number
    
    Returns:
    int : Sum of a and b
    """
    return a + b

def subtract(a: int, b: int) -> int:
    """Subtract b from a.
    
    Args:
    a : First number
    b : Second number
    
    Returns:
    int : Difference between a and b
    """
    return a - b

def multiply(a: int, b: int) -> int:
    """Multiple b from a.
    
    Args:
    a : First number
    b : Second number
    
    Returns:
    int : Product of a and b
    """
    return a * b

def divide(a: int, b: int) -> int:
    """Divide b from a.
    
    Args:
    a : First number
    b : Second number
    
    Returns:
    int : Divide of a and b
    """
    return a / b

tools = [add, subtract, multiply, divide]

In [ ]:
## LLM with tools
llm_with_tools = llm.bind_tools(tools, parallel_tool_calls = False) ## parallel_tool_call = false -> For sequence execution

In [ ]:
## Defining the state
# from langgraph.graph import MessagesState
# or 
from typing_extensions import TypedDict
from langchain_core.messages import AnyMessage
from typing import Annotated
from langgraph.graph.message import add_messages

class MessageState(TypedDict):
    messages:Annotated[list[AnyMessage],add_messages] ## reducer function

In [ ]:
## Adding Prompts 
from langchain_core.messages import HumanMessage, SystemMessage 
sys_msg = SystemMessage(content="You are a helpful assistant tasked with performing arithmetic on a set of inputs.")

In [ ]:
## Creating Arithmeric Assistant 
def assistant(state:MessageState):
    return {"messages":[llm_with_tools.invoke([sys_msg] + state["messages"])]}

In [ ]:
## Creating Graph 
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition

builder=StateGraph(MessageState)

# Defining the node
builder.add_node("assistant",assistant)
builder.add_node("tools",ToolNode(tools))

## Defining the edges 
builder.add_edge(START, "assistant")
builder.add_conditional_edges("assistant", tools_condition, )
## If the latest messages (result) from assistant is a tool call -> tools_condition routes to tools 
## If the latest messages from assistant is a tool call -> tools_condition routes to tools 

builder.add_edge("tools", "assistant")
react_graph = builder.compile()

## Displaying Graph 
display(Image(react_graph.get_graph().draw_mermaid_png()))

In [ ]:
messages = [HumanMessage(content="Add 10 and 14")]
messages = react_graph.invoke({"messages": messages})
for m in messages['messages']:
    m.pretty_print()

In [ ]:
messages = [HumanMessage(content="Hello, what is 2 multiplied by 2 then plus 2 then add 4?")]
messages = react_graph.invoke({"messages": messages})
for m in messages['messages']:
    m.pretty_print()

In [ ]:
messages = [HumanMessage(content="Add 10 and 14. Multiply the output by 2. Divide the output by 5")]
messages = react_graph.invoke({"messages": messages})
for m in messages['messages']:
    m.pretty_print()

## Agents Memory -> MemorySaver

In [ ]:
from langgraph.checkpoint.memory import MemorySaver 
memory = MemorySaver()
react_graph = builder.compile(checkpointer=memory)

In [ ]:
config = {"configurable" : {"thread_id" : 1}}

messages = [HumanMessage(content="What is 3 and 4?")]
messages = react_graph.invoke({"messages": messages}, config)
for m in messages['messages']:
    m.pretty_print()

In [ ]:
messages = [HumanMessage(content="Then add 10")]
messages = react_graph.invoke({"messages": messages}, config)
for m in messages['messages']:
    m.pretty_print()